<a href="https://colab.research.google.com/github/ivanpalenciab/Sentiment-analysis/blob/data_preparation/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%pip install pyarrow
%pip install transformers

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [7]:
RANDOM_SEED = 30
MAX_LEN = 200
FILE_ROUTE = "/content/drive/MyDrive/Sentiment analysis/Datos/"
BATCH_SIZE = 16
NCLASSES = 2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [38]:
df_train= pd.read_parquet(f"{FILE_ROUTE}spanish%2Fmultilingual-sentiments-train.parquet")
df_train["label"] = df["label"].replace({0: 1, 1: 0, 2: -1})
df_train.drop("source",axis=1,inplace=True)
df_train.head(10)

,text,label
0,estoy hasta el ojete de que me digáis que teng...,-1
1,@user Por? Tenía pensado verla después de la ...,1
2,Esto de estar feliz mola,0
3,Ya no es tan divertido,-1
4,@user te recuerdo que soy una persona que tien...,1
5,@user @user @user con una pequeña donación har...,0
6,He probado una nueva espuma para el pelo y sí ...,-1
7,@user Ojalá pudiera darte el abrazo en vivo o ...,1
8,@user aquí tienes a mi bae aka una egipcia pre...,0
9,"@user ya somos dos, que triste",-1


In [39]:
df_train["label"].unique()

array([-1,  1,  0])

In [40]:
test_df = pd.read_parquet(f"{FILE_ROUTE}spanish%2Fmultilingual-sentiments-test.parquet")
test_df["label"] = test_df["label"].replace({0: 1, 1: 0, 2: -1})
test_df.drop("source",axis=1,inplace=True)
test_df.head()

,text,label
0,"@user jajajaja dale, hacete la boluda vos jaja...",-1
1,cada vez que cito un tweet se va la ubicación ...,0
2,@user MAAAAE RAJADO! Pero lo bueno es q uno se...,1
3,Bueno hoy fui a almorzar a Nanay con otras 3 d...,-1
4,Necesito seguir a mas cuentas camren shippers ...,0


In [20]:
#tokenitation
PRE_TRAINED_MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [22]:
# tokenitation example
sample_txt = '¡realmente me gusta la hamburguesa!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Frase:  ¡realmente me gusta la hamburguesa!
Tokens:  ['¡', 'realmente', 'me', 'gusta', 'la', 'hamburguesa', '!']
Tokens numéricos:  [1139, 3189, 1129, 2816, 1030, 27035, 1127]


In [29]:
# Codificactio to introduce in BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 11,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    padding=True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [30]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [31]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', '¡', 'realmente', 'me', 'gusta', 'la', 'hamburguesa', '!', '[SEP]']
tensor([    4,  1139,  3189,  1129,  2816,  1030, 27035,  1127,     5])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])


In [36]:
#dataset creation
class my_dataset(Dataset):
  def __init__(self,text,labels,tokenizer,max_len):
    self.text = text
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.text)

  def __getitem__(self, item):
    text = str(self.text[item])
    label = self.label[item]
    encoding = tokenizer.encode_plus(
        text,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        padding=True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )


    return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      }


In [42]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = my_dataset(
      text = df.text.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [43]:
train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
